In [ ]:
#imports: do not change them
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
import torch.nn as nn 
from PIL import Image
from torchvision import transforms 
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torchvision
import random
from tqdm import tqdm
from torch.utils.data import random_split
from torch.autograd import Variable
import cv2
import torchvision.utils as utils

## Please DONOT remove these lines. 
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

### Question 1: MultiHead-Attention
Given a set of query, key, value vectors, calculate the attention value using multihead attention.

choose the closest option to the maximum value of the attention vector


1.   0.5590
2.   0.7851
3.   0.2312
4.   0.9452

Answer (1)

In [ ]:
## Please DO NOT change these values.
#This is the config that will be used for implementing the Multi-head attention 
config = {
    'QI_DIM': 64,
    'VI_DIM': 64,
    'KI_DIM': 64,
    'QO_DIM': 32,
    'VO_DIM': 32,
    'OP_DIM': 32,
    'NUM_HEADS': 8,
}

In [ ]:
#A class for implementing the Multi-head Attention
class MultiHeadAttention(nn.Module):

    def __init__(self, config):
        super(MultiHeadAttention, self).__init__()

        ### YOUR CODE STARTS HERE ###

        #set the op dimension, number of heads based on the value given in config
        self.op_dim = config['OP_DIM']
        self.n_heads = config['NUM_HEADS']

        #set the query dimension as QO_DIM from the config and value dimension as VO_DIM
        self.query_dim = config['QO_DIM']
        self.value_dim = config['VO_DIM']

        #set the respective dimensions
        self.QTrans = nn.Linear(config['QI_DIM'], config['QO_DIM'])
        self.KTrans = nn.Linear(config['KI_DIM'], config['QO_DIM'])
        self.VTrans = nn.Linear(config['VI_DIM'], config['VO_DIM'])
        self.OTrans = nn.Linear(config['VO_DIM'], config['OP_DIM'])

        #set the scaling factor as follows:
        #scale = sqrt(QO_DIM/NUM_HEADS)
        self.scale = torch.sqrt(torch.FloatTensor([self.query_dim // self.n_heads]))

        ### YOUR CODE ENDS HERE ###

    
    
    def forward(self, q_inp, k_inp, v_inp, ret_atn=False):

        #making sure that shapes are similar across dim=0
        assert q_inp.shape[0] == k_inp.shape[0]

        #setting batch size
        batch_size = q_inp.shape[0]
        seq_length = q_inp.shape[1]

        ### YOUR CODE STARTS HERE ###
        #pass q_inp, k_inp, v_inp through the linear layers defined in the
        #init function
        
        Q = self.QTrans(q_inp)
        K = self.KTrans(k_inp)
        V = self.VTrans(v_inp)

        #reshape the Q, K, V tensors to (batch_size, num_heads, -1, self.query_dim/num_heads)
        #hint: you can also do the same by the view() of pytorch
        Q = Q.view(batch_size, -1, self.n_heads, self.query_dim // self.n_heads).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.query_dim // self.n_heads).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.value_dim // self.n_heads).permute(0, 2, 1, 3)
        

        # define energy as matrix product of Q and K divided by self.scale value
        #hint: use torch.matmul() for this and do not forget to reshape K vector as (batch_size, num_heads, self.query_dim/num_heads, -1)
        #this should be done in order to carry out the matrix multiplication
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        #define attention as softmax of energy vector across the last dimension
        attention = torch.softmax(energy, dim = -1)

        ### YOUR CODE ENDS HERE ###

        #multiplying the attention with value tensor
        attended = torch.matmul(attention, V).permute(0, 2, 1, 3).contiguous()

        #passing the attended tensor through OTrans linear layer
        out = self.OTrans(attended.view(batch_size, seq_length, -1))
        
        if ret_atn:
            return out, attention
        
        else:
            return out

In [ ]:
## Please DONOT remove these lines. 
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

q = torch.rand(4, 2, 64)
k = torch.rand(4, 2, 64)
v = torch.rand(4, 2, 64)
## Please DONOT remove these lines. 

### YOUR CODE STARTS HERE ###

#initialise an instance of MultiHeadAttention class
d = MultiHeadAttention(config)

#pass q, k, v values through the instance
x, y = d(q, k, v, ret_atn = True)

#print the max value of the attention and report your answer
torch.max(y)

### YOUR CODE ENDS HERE ###


tensor(0.5590, grad_fn=<MaxBackward1>)

### Question 2: Learning to pay attention
For questions 2 & 3, we will implement the paper "Learning to pay Attention", published at ICLR2018. The paper focuses on Trainable Soft Visual Attention in CNNs for image classification task. Follow the steps given in the upcoming code blocks to answer question 2.

What is the range of the mean value obtained?


1.   0.01 - 0.1
2.   0.1 - 0.3
3.   0.5 - 0.8
4.   1 - 1.5

Answer (2)

In [ ]:
#defining a block that has conv->batchnorm->relu
class ConvBlock(nn.Module):
    def __init__(self, in_features, out_features, num_conv, pool=False):
        super(ConvBlock, self).__init__()
        features = [in_features] + [out_features for i in range(num_conv)]

        #appending all the operations in the list and then converting them to sequential set of operations
        layers = []
        for i in range(len(features)-1):
            layers.append(nn.Conv2d(in_channels=features[i], out_channels=features[i+1], kernel_size=3, padding=1, bias=True))
            layers.append(nn.BatchNorm2d(num_features=features[i+1], affine=True, track_running_stats=True))
            layers.append(nn.ReLU())
            if pool:
                layers.append(nn.MaxPool2d(kernel_size=2, stride=2, padding=0))
        self.op = nn.Sequential(*layers)
    def forward(self, x):
        return self.op(x)

#defining a projector block which is nothing but a conv layer that is used to transform the channel dimension
class ProjectorBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ProjectorBlock, self).__init__()
        self.op = nn.Conv2d(in_channels=in_features, out_channels=out_features, kernel_size=1, padding=0, bias=False)
    def forward(self, inputs):
        return self.op(inputs)


#defining a linear attention block as mentioned in the paper
class LinearAttentionBlock(nn.Module):
    def __init__(self, in_features, normalize_attn=True):
        super(LinearAttentionBlock, self).__init__()

        ### YOUR CODE STARTS HERE ###
        #set self.mormalize_attn as the normalize_attn argument
        self.normalize_attn = normalize_attn

        #define self.op as a conv layer with in_channels=in_features, out_channels=1, kernel_size=1, padding=0 and with bias=False
        self.op = nn.Conv2d(in_channels=in_features, out_channels=1, kernel_size=1, padding=0, bias=False)

        ### YOUR CODE ENDS HERE ###


    def forward(self, l, g):
        ### YOUR CODE STARTS HERE ###

        #set N, C, W, H using the size of the 'l' tensor
        #where N = number of batches, C = channels, H = height of the tensor, W = width of the tensor
        N, C, W, H = l.size()

        #add l and g as mentioned in the paper and pass it through the self.op layer
        c = self.op(l+g) # batch_sizex1xWxH

        #pass the output of the self.op layer through sigmoid activation function
        a = torch.sigmoid(c)

        #make the size of a same as l
        #hint: you can do this by the tensor.expand_as() method of PyTorch

        #multiple a and l using torch.mul()
        g = torch.mul(a.expand_as(l), l)

        ### YOUR CODE ENDS HERE ###

        #using adaptive avg pooling as the final operation
        g = F.adaptive_avg_pool2d(g, (1,1)).view(N,C)

        return c.view(N,1,W,H), g


In [ ]:
## Please DONOT remove/change these lines. 
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)
att = LinearAttentionBlock(in_features=16, normalize_attn=True)

l = torch.rand(4, 16, 64, 64)
g = torch.rand(4, 16, 64, 64)
## Please DONOT remove/change these lines. 


### YOUR CODE STARTS HERE ###
#pass the l and g vectors through the attention instance
d, f = att(l, g)
#take the mean of all the elements present in the output 'g' vector and mark your answer for question 2
#hint: use torch.mean()
val = torch.mean(f)
print(val)
### YOUR CODE ENDS HERE ###

tensor(0.1692, grad_fn=<MeanBackward0>)


### Question 3: Training the Attention VGGNet
Follow the steps in the below given code blocks to build and train the Attention VGGNet model on CIFAR100 dataset.

What is the range of the test set accuracy obtained after training the model for 5 epochs?


1.   0 - 2%
2.   2 - 10%
3.   40 - 50%
4.   25 - 30%

Answer (2)

In [ ]:
#building a VGGNet integrated with the attention implemented in the above code block
class AttnVGG_before(nn.Module):
    def __init__(self, im_size, num_classes, attention=True, normalize_attn=True):
        super(AttnVGG_before, self).__init__()
        self.attention = attention
        # defining conv blocks with certain choices of input and output features
        #this set of choices is taking from the VGG style model
        self.conv_block1 = ConvBlock(3, 64, 2)
        self.conv_block2 = ConvBlock(64, 128, 2)
        self.conv_block3 = ConvBlock(128, 256, 3)
        self.conv_block4 = ConvBlock(256, 512, 3)
        self.conv_block5 = ConvBlock(512, 512, 3)
        self.conv_block6 = ConvBlock(512, 512, 2, pool=True)
        self.dense = nn.Conv2d(in_channels=512, out_channels=512, kernel_size=int(im_size/32), padding=0, bias=True)

        # defining the projector layers and the multiple attention blocks as mentioned in the paper
        if self.attention:
            self.projector = ProjectorBlock(256, 512)
            self.attn1 = LinearAttentionBlock(in_features=512, normalize_attn=normalize_attn)
            self.attn2 = LinearAttentionBlock(in_features=512, normalize_attn=normalize_attn)
            self.attn3 = LinearAttentionBlock(in_features=512, normalize_attn=normalize_attn)

        # final classification layer
        if self.attention:
            self.classify = nn.Linear(in_features=512*3, out_features=num_classes, bias=True)
        else:
            self.classify = nn.Linear(in_features=512, out_features=num_classes, bias=True)

    def forward(self, x):

        ### YOUR CODE STARTS HERE ###

        # passing the input through all the defined layers
        #pass x through conv_block1 and collect the output in variable name x
        x = self.conv_block1(x)

        #pass x through conv_block2 and collect the output in variable name x
        x = self.conv_block2(x)

        #pass x through conv_block3 and collect the output in variable name l1
        l1 = self.conv_block3(x) 

        #pass l1 through max pool 2d and collect the output in variable name x
        x = F.max_pool2d(l1, kernel_size=2, stride=2, padding=0) # /2

        #pass x through conv_block4 and collect the output in av ariable called l2
        l2 = self.conv_block4(x) # /2

        #pass l2 through max pool 2d and collect the output in a variable called x
        x = F.max_pool2d(l2, kernel_size=2, stride=2, padding=0) # /4

        #pass x through conv_block5 and collect the output in a variable called l3
        l3 = self.conv_block5(x) # /4

       #pass l3 through max pool 2d and collect the output in variable name x
        x = F.max_pool2d(l3, kernel_size=2, stride=2, padding=0) # /8

        #pass x through conv_block6 and collect the output in a variable called x
        x = self.conv_block6(x) # /32

        #pass x through self.dense and collect the output in a variable called g
        g = self.dense(x) # batch_sizex512x1x1

        # attention calculations are carried out according to the paper
        if self.attention:

            #pass l1 through self.projector and collect the output in variable t
            #pass g and t through self.attn1 and collect the output in variable names c1 and g1 respectively
            c1, g1 = self.attn1(self.projector(l1), g)

            #pass l2 and g through self.attn2 and collect the output in variable names c2 and g2 respectively
            c2, g2 = self.attn2(l2, g)

            #pass l3 and g through self.attn3 and collect the output in variable names c3 and g3 respectively
            c3, g3 = self.attn3(l3, g)

            #concat g1, g2 and g3 across dim=1 and collect the output in a variable called g
            #hint:use torch.cat()
            g = torch.cat((g1,g2,g3), dim=1)

            # pass g through self.classify layer and collect the output in a variable called x
            x = self.classify(g)

        ### YOUR CODE ENDS HERE ###

        #if attention is set to false
        else:
            c1, c2, c3 = None, None, None
            x = self.classify(torch.squeeze(g))

        return [x, c1, c2, c3]

In [ ]:
#defining arg parse and running it
parser = argparse.ArgumentParser()

parser.add_argument("--batch_size", type=int, default=128, help="batch size")
parser.add_argument("--epochs", type=int, default=5, help="number of epochs")
parser.add_argument("--lr", type=float, default=0.1, help="initial learning rate")
parser.add_argument("-f", "--file", required=False) 


opt = parser.parse_args()

In [ ]:
#defining the main function for training
def main():
    print('\nloading the dataset ...\n')

    #setting the image size to 32 as we will be using CIFAR100 dataset
    im_size = 32

    #defining train transforms
    transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
    ])
    
    #defining the trainset from the torchvision library
    trainset = torchvision.datasets.CIFAR100(root='CIFAR100_data', train=True, download=True, transform=transform_train)
    
    #we will be training on a random 5000 image subset of the training data to save time
    sub_train_size = 5000
    train_size = len(trainset) - sub_train_size
    extra_ds, train_ds = random_split(trainset, [train_size, sub_train_size], generator=torch.Generator().manual_seed(30))

    #defining the dataloader
    trainloader = torch.utils.data.DataLoader(train_ds, batch_size=opt.batch_size, shuffle=False, num_workers=0, worker_init_fn = lambda id: np.random.seed(id))
    # testset = torchvision.datasets.CIFAR100(root='CIFAR100_data', train=False, download=False, transform=transform_test)
    # testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=0, worker_init_fn = lambda id: np.random.seed(id))
    print('done')

    print('\npay attention before maxpooling layers...\n')
    #calling the model that needs to be trained
    net = AttnVGG_before(im_size=im_size, num_classes=100,
        attention=True, normalize_attn=True)
    
    #setting the loss function to CE loss
    criterion = nn.CrossEntropyLoss()
    print('done')

    #setting the device to GPU and transfering model and loss function tot he device
    print('\nmoving to GPU ...\n')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = net.to(device)
    criterion.to(device)
    print('done')

    # defining the optimizer
    optimizer = optim.SGD(model.parameters(), lr=opt.lr, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

    # training loop starts here
    print('\nstart training ...\n')
    step = 0

    #looping over the total number of epochs
    for epoch in range(opt.epochs):
        
        # adjusting the learning rate
        scheduler.step()

        print("\nepoch %d learning rate %f\n" % (epoch, optimizer.param_groups[0]['lr']))

        # looping for every epoch
        for i, data in enumerate(trainloader, 0):
            model.train()
            model.zero_grad()
            optimizer.zero_grad()
            inputs, labels = data

            #transfering the data to device
            inputs, labels = inputs.to(device), labels.to(device)

            # forward passing the inputs to the model
            pred, __, __, __ = model(inputs)

            # backwardpass for loss calculation
            loss = criterion(pred, labels)
            loss.backward()
            optimizer.step()

            # display results for every 3 iterations
            if i % 3 == 0:
                model.eval()
                pred, __, __, __ = model(inputs)
                predict = torch.argmax(pred, 1)
                total = labels.size(0)
                correct = torch.eq(predict, labels).sum().double().item()
                accuracy = correct / total
                print("[epoch %d][%d/%d] loss %.4f accuracy %.2f%%"
                    % (epoch, i, len(trainloader)-1, loss.item(), (100*accuracy)))
            step += 1

    #saving the weights of the model at the end of total number of epochs
    torch.save(model.state_dict(), 'net.pth')

In [ ]:
#please change the runtime type to GPU before running this cell
#follow the instructions given in this blog for more details: https://www.tutorialspoint.com/google_colab/google_colab_using_free_gpu.htm
main()


loading the dataset ...



  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting CIFAR100_data/cifar-100-python.tar.gz to CIFAR100_data
done

pay attention before maxpooling layers...

done

moving to GPU ...

done

start training ...


epoch 0 learning rate 0.095000



/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[epoch 0][0/39] loss 4.6173 accuracy 1.56%
[epoch 0][3/39] loss 4.6021 accuracy 1.56%
[epoch 0][6/39] loss 4.5919 accuracy 0.78%
[epoch 0][9/39] loss 4.5264 accuracy 0.00%
[epoch 0][12/39] loss 4.4593 accuracy 0.78%
[epoch 0][15/39] loss 4.5684 accuracy 4.69%
[epoch 0][18/39] loss 4.5032 accuracy 3.91%
[epoch 0][21/39] loss 4.3271 accuracy 4.69%
[epoch 0][24/39] loss 4.2961 accuracy 3.91%
[epoch 0][27/39] loss 4.4167 accuracy 0.00%
[epoch 0][30/39] loss 4.5069 accuracy 2.34%
[epoch 0][33/39] loss 4.4371 accuracy 1.56%
[epoch 0][36/39] loss 4.4722 accuracy 0.78%
[epoch 0][39/39] loss 4.1007 accuracy 25.00%

epoch 1 learning rate 0.090250

[epoch 1][0/39] loss 4.3485 accuracy 3.91%
[epoch 1][3/39] loss 4.3648 accuracy 3.12%
[epoch 1][6/39] loss 4.3658 accuracy 4.69%
[epoch 1][9/39] loss 4.3276 accuracy 3.12%
[epoch 1][12/39] loss 4.3792 accuracy 3.91%
[epoch 1][15/39] loss 4.2496 accuracy 5.47%
[epoch 1][18/39] loss 4.3291 accuracy 3.91%
[epoch 1][21/39] loss 4.2954 accuracy 4.69%
[epoch

In [ ]:
transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
    ])
testset = torchvision.datasets.CIFAR100(root='CIFAR100_data', train=False, download=False, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=0, worker_init_fn = lambda id: np.random.seed(id))
model = AttnVGG_before(im_size=32, num_classes=100, attention=True, normalize_attn=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('/content/net.pth', map_location=torch.device("cuda")))
model.to(device)
model.eval()
total = 0
correct = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        images_test, labels_test = data
        images_test, labels_test = images_test.to(device), labels_test.to(device)
        pred_test, __, __, __ = model(images_test)
        predict = torch.argmax(pred_test, 1)
        total += labels_test.size(0)
        correct += torch.eq(predict, labels_test).sum().double().item()
    print("\nAccuracy on test data: %.2f%%\n" % (100*correct/total))


Accuracy on test data: 6.03%

